In [25]:
import numpy as np  # 引入numpy、pandas库
import pandas as pd
from dbfread import DBF
from pandas import DataFrame

# def count_storms_times_year(path):
#     table = DBF(path)
#     df = pd.DataFrame(iter(table))
#     row_size = df.shape[0] 
#     temp1 = df # 复制dataframe表格 避免修改原始数据表格
#     ls = [] # 新建每一年台风出现的次数
#     for i in range(1980, 2022):  # 遍历1980到2021年台风
#         temp = temp1.loc[temp1['SEASON']==i] # 遍历第i年台风
#         counts = temp['NUMBER'].value_counts() # 统计每个台风出现的次数
#         ls.append(counts)
#     number_times = []
#     for i in range(len(ls)):
#         number_times.append(ls[i].size)
#     return number_times

path = "./SI.dbf"
# path = "./SP.dbf"


df = pd.DataFrame(iter(DBF(path)))
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

In [14]:
# def classification(typh_intensity):
#     if typh_intensity >= 1 and typh_intensity <= 44:
#         result = 'D'
#     elif typh_intensity > 44 and typh_intensity <= 63:
#         result = 'S'
#     elif typh_intensity > 63 and typh_intensity <= 82:
#         result = 'C1'
#     elif typh_intensity > 82 and typh_intensity <= 95:
#         result = 'C2'
#     elif typh_intensity > 95 and typh_intensity <= 113:
#         result = 'C3'
#     elif typh_intensity > 113 and typh_intensity <= 135:
#         result = 'C4'        
#     elif typh_intensity > 135:
#         result = 'C5'
#     else:
#         result = "WRONG"
#     return result

def classification(typh_intensity):
    if typh_intensity >= 1 and typh_intensity <= 33:
        result = 'D'
    elif typh_intensity > 33 and typh_intensity <= 63:
        result = 'S'
    elif typh_intensity > 63 and typh_intensity <= 82:
        result = 'C1'
    elif typh_intensity > 82 and typh_intensity <= 95:
        result = 'C2'
    elif typh_intensity > 95 and typh_intensity <= 112:
        result = 'C3'
    elif typh_intensity > 112 and typh_intensity <= 136:
        result = 'C4'        
    elif typh_intensity > 136:
        result = 'C5'
    else:
        result = "WRONG"
    return result

def year(df, YEAR):
    df4 = df[df["SEASON"] == YEAR]
    typhoon_time = df4['NUMBER'].value_counts()  # 统计每个台风出现的次数
    number_typhoon = typhoon_time[:].index[:].tolist() # 统计台风的编号
    number_typhoon.sort() # 对编号进行排序
    ls_max_intensity = []
    ls_classification = []  # 分类列表
    for index in range(len(number_typhoon)):
        df5 = df4[df4["NUMBER"] == number_typhoon[index]]
        index_max = df5['USA_WIND'].max()
        classifation_result = classification(index_max)
        ls_classification.append(classifation_result)
        ls_max_intensity.append(index_max)
    
    ls_year = []
    for i in range(len(number_typhoon)):
        ls_year.append(YEAR)
    tc0 = pd.DataFrame(data= ls_year, columns=['year'] )
    tc1 = pd.DataFrame(data = number_typhoon, columns = ['typhoon_index'])
    tc2 = pd.DataFrame(data = ls_max_intensity, columns = ['max_intensity'])
    tc3 = pd.DataFrame(data = ls_classification, columns = ['typhoon_classification'])
    result_chart = pd.concat([tc0, tc1, tc2, tc3], axis = 1)
    return result_chart

def merge(df, start_year, end_year):  # 合并成一个Dataframe文件
    new_chart = pd.DataFrame(columns=['year', 'typhoon_index', 'max_intensity', 'typhoon_classification'],index = None)
    for years in range(start_year, end_year+1):
        year_chart = year(df, years)
        new_chart = new_chart.append(year_chart, ignore_index=True)
    return new_chart

def reindex(classification_times):
    return classification_times.reindex(['D', 'S', 'C1', 'C2', 'C3', 'C4', 'C5', 'WRONG'])

def year_classification(class_chart, years):
    year_classification = class_chart.loc[class_chart['year'] == years]
    year_classification = year_classification.typhoon_classification.value_counts()
    year_classification = reindex(year_classification)
    year_classification = pd.DataFrame(year_classification)
    return year_classification

In [26]:
frequency = pd.DataFrame(columns=[ 'D', 'S', 'C1', 'C2', 'C3', 'C4', 'C5', 'WRONG'],index = None)
for i in range(1897, 2022):
    df_year2next = df[((df['year'] == i) & (df['month'] >= 7)) 
   | ((df['year'] == i+1) & (df['month'] <= 6))]
    chart = merge(df_year2next, i, i+1)
    tmp_count = chart.typhoon_classification.value_counts()
    tmp_cla = tmp_count.reindex(['D', 'S', 'C1', 'C2', 'C3', 'C4', 'C5', 'WRONG'], fill_value=0)
    tmp_cla_year = pd.DataFrame(tmp_cla).T
    tmp_year = tmp_cla_year.rename(index={"typhoon_classification": str(i)+'to'+str(i+1)})
    frequency = frequency.append(tmp_year)

frequency

,D,S,C1,C2,C3,C4,C5,WRONG
1897to1898,0,0,0,0,0,0,0,17
1898to1899,0,0,0,0,0,0,0,11
1899to1900,0,0,0,0,0,0,0,13
1900to1901,0,0,0,0,0,0,0,14
1901to1902,0,0,0,0,0,0,0,19
1902to1903,0,0,0,0,0,0,0,32
1903to1904,0,0,0,0,0,0,0,10
1904to1905,0,0,0,0,0,0,0,20
1905to1906,0,0,0,0,0,0,0,14
1906to1907,0,0,0,0,0,0,0,20


In [27]:
frequency = frequency.drop('WRONG', axis=1)

In [28]:
frequency['Row_sum'] = frequency.apply(lambda x: x.sum(), axis=1) # 按行求和，添加为新列


In [29]:
frequency

,D,S,C1,C2,C3,C4,C5,Row_sum
1897to1898,0,0,0,0,0,0,0,0
1898to1899,0,0,0,0,0,0,0,0
1899to1900,0,0,0,0,0,0,0,0
1900to1901,0,0,0,0,0,0,0,0
1901to1902,0,0,0,0,0,0,0,0
1902to1903,0,0,0,0,0,0,0,0
1903to1904,0,0,0,0,0,0,0,0
1904to1905,0,0,0,0,0,0,0,0
1905to1906,0,0,0,0,0,0,0,0
1906to1907,0,0,0,0,0,0,0,0


In [31]:
frequency.to_excel("./SI.xlsx")